# 1. Preprocessing
## Import data

In [1]:
! pip install scikeras
! pip install pandas
! pip install nltk
! pip install scikit-learn



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
df = pd.read_csv('training.300000.processed.noemoticon.csv', encoding='ISO-8859-1')

## Text clean

In [2]:
import nltk
nltk.download("stopwords")
nltk.download("punkt")
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /home/liu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/liu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/liu/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [3]:
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))


def clean_text(text):
    # transfer to lower characters
    text = text.lower()
    
    # eliminate url link
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # eliminate special character
    text = re.sub(r'@\w+|#\w+', '', text)
    
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # eliminate stop word
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    
   
    return ' '.join(tokens)

# apply clean function to data frame
df['cleaned_text'] = df['text'].apply(clean_text)

df[['text', 'cleaned_text']].head()

,text,cleaned_text
0,@chrishasboobs AHHH I HOPE YOUR OK!!!,ahhh hope ok
1,"@misstoriblack cool , i have no tweet apps fo...",cool tweet apps razr
2,@TiannaChaos i know just family drama. its la...,know family drama lamehey next time u hang kim...
3,School email won't open and I have geography ...,school email wont open geography stuff revise ...
4,upper airways problem,upper airways problem


## Tokenization, Padding

In [19]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


# Define tokenizer
tokenizer = Tokenizer(num_words=10000)


tokenizer.fit_on_texts(df['cleaned_text'])

# Transfer to sequences
sequences = tokenizer.texts_to_sequences(df['cleaned_text'])


# Padding sequence to a constant length
X = pad_sequences(sequences, maxlen=50)

# Transfer original labels to 0-1 label
y = df['sentiment'].values  
y = (y == 4).astype(int) 


## Divide dataset into two parts, trainset and testset

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Deep learning

## CNN
Model implement

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Embedding, Dropout, Flatten, Input

# from previous block, which maximum length of sequence is 50
sequence_length = 50

# define cnn
model_1 = Sequential()

# add input layer
model_1.add(Input(shape=(sequence_length,)))

# add embedding layer, map word id to a vector
model_1.add(Embedding(input_dim=10000, output_dim=50))

# convolutional layer
model_1.add(Conv1D(filters=128, kernel_size=5, activation='relu'))

# pool layer
model_1.add(MaxPooling1D(pool_size=2))

# fc1
model_1.add(Flatten())

# Dropout layer
model_1.add(Dropout(0.5))

# fc2
model_1.add(Dense(64, activation='relu'))

# output layer
model_1.add(Dense(1, activation='sigmoid'))

model_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# show model structure
model_1.summary()

model_1.fit(X_train, y_train, epochs=10, batch_size=128, validation_split=0.2)
loss, accuracy = model_1.evaluate(X_test, y_test)


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)         │ (None, 50, 50)         │       500,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 46, 128)        │        32,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 23, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2944)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 2944)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 64)             │       188,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 720,673 (2.75 MB)

 Trainable params: 720,673 (2.75 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 21s 13ms/step - accuracy: 0.7069 - loss: 0.5457 - val_accuracy: 0.7729 - val_loss: 0.4747
Epoch 2/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 19s 13ms/step - accuracy: 0.7935 - loss: 0.4452 - val_accuracy: 0.7721 - val_loss: 0.4754
Epoch 3/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 20s 14ms/step - accuracy: 0.8187 - loss: 0.3998 - val_accuracy: 0.7664 - val_loss: 0.4918
Epoch 4/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 21s 14ms/step - accuracy: 0.8418 - loss: 0.3565 - val_accuracy: 0.7622 - val_loss: 0.5246
Epoch 5/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 19s 13ms/step - accuracy: 0.8633 - loss: 0.3139 - val_accuracy: 0.7568 - val_loss: 0.5788
Epoch 6/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 18s 12ms/step - accuracy: 0.8779 - loss: 0.2805 - val_accuracy: 0.7506 - val_loss: 0.6524
Epoch 7/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 18s 12ms/step - accuracy: 0.8919 - loss: 0.2495 - val_accuracy: 0.7494 - val_loss: 0.7186
Epoch 8/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 19s 12ms/step - accuracy: 0.9001 -

Fine tune

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Embedding, Dropout, Flatten, Input
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
import json


# Define cnn model structure
def create_model(optimizer='adam', filters=64, kernel_size=5, dropout_rate=0.5):
    sequence_length = 50  
    model = Sequential()
    model.add(Input(shape=(sequence_length,)))
    model.add(Embedding(input_dim=10000, output_dim=50))
    model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dropout(dropout_rate))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    return model
# Wrape Keras model for sklearn
model_1_f = KerasClassifier(build_fn=create_model, epochs=10, batch_size=128, verbose=0)

# Search list
param_dist = {
    'model__optimizer': ['adam', 'rmsprop'],
    'model__filters': [32, 64, 128],
    'model__kernel_size': [3, 5],
    'model__dropout_rate': [0.2, 0.3],
    'epochs': [5, 10, 15],
    'batch_size': [32, 64]
}

random_search = RandomizedSearchCV(estimator=model_1_f, param_distributions=param_dist, n_iter=10, cv=3)
random_search.fit(X_train,y_train)
# save best paramaters combination
best_params = random_search.best_params_
with open('best_paramscnn.json', 'w') as f:
    json.dump(best_params, f)
#    
model_1_best=random_search.best_estimator_.model_
model_1_best.save("bestcnn.h5")


/home/liu/miniconda3/envs/tensorflow/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/home/liu/miniconda3/envs/tensorflow/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/home/liu/miniconda3/envs/tensorflow/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/home/liu/miniconda3/envs/tensorflow/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will 

RandomizedSearchCV(cv=3,
                   estimator=KerasClassifier(batch_size=128, build_fn=<function create_model at 0x7fce7c740e00>, epochs=10, verbose=0),
                   param_distributions={'batch_size': [32, 64],
                                        'epochs': [5, 10, 15],
                                        'model__dropout_rate': [0.2, 0.3],
                                        'model__filters': [32, 64, 128],
                                        'model__kernel_size': [3, 5],
                                        'model__optimizer': ['adam',
                                                             'rmsprop']})

In [21]:
import json
from sklearn.metrics import accuracy_score

# 获取最佳参数
best_params = random_search.best_params_

# 将最佳参数保存为 JSON 文件
with open('best_paramscnn.json', 'w') as f:
    json.dump(best_params, f)

# 打印最佳参数
print(f"Best Parameters: {best_params}")

# 使用最佳模型对测试集进行评估
model_1_best = random_search.best_estimator_

# 评估模型并打印结果（例如使用准确率评估）
accuracy = model_1_best.score(X_test,y_test)
model_1_best=random_search.best_estimator_.model_
loss, accuracy= model_1_best.evaluate(X_test, y_test)
print(loss,accuracy)
model_1_best.save("bestcnn.h5")


Best Parameters: {'model__optimizer': 'rmsprop', 'model__kernel_size': 3, 'model__filters': 32, 'model__dropout_rate': 0.2, 'epochs': 15, 'batch_size': 32}
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7760 - loss: 0.4878


0.49118101596832275 0.777233362197876


## LSTM
model implement

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout, Input
from tensorflow.keras.optimizers import Adam
def create_lstm_model(optimizer='adam', dropout_rate=0.5):
    model=Sequential()
    sequence_length=50
    model.add(Input(shape=(sequence_length,)))
    model.add(Embedding(input_dim=10000, output_dim=50))
    model.add(LSTM(units=128, return_sequences=False))
    model.add(Dropout(dropout_rate))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    return model

model_2=create_lstm_model()
model_2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_2.summary()
model_2.fit(X_train,y_train,epochs=10, batch_size=128, validation_split=0.2)
loss, accuracy=model_2.evaluate(X_test,y_test)
    
    

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_6 (Embedding)         │ (None, 50, 50)         │       500,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_8 (LSTM)                   │ (None, 128)            │        91,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 599,969 (2.29 MB)

 Trainable params: 599,969 (2.29 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 28s 18ms/step - accuracy: 0.7154 - loss: 0.5422 - val_accuracy: 0.7724 - val_loss: 0.4807
Epoch 2/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 28s 18ms/step - accuracy: 0.7880 - loss: 0.4493 - val_accuracy: 0.7739 - val_loss: 0.4748
Epoch 3/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 28s 19ms/step - accuracy: 0.8026 - loss: 0.4230 - val_accuracy: 0.7720 - val_loss: 0.4835
Epoch 4/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 29s 19ms/step - accuracy: 0.8139 - loss: 0.3999 - val_accuracy: 0.7686 - val_loss: 0.4907
Epoch 5/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 30s 20ms/step - accuracy: 0.8229 - loss: 0.3814 - val_accuracy: 0.7666 - val_loss: 0.5140
Epoch 6/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 28s 19ms/step - accuracy: 0.8360 - loss: 0.3570 - val_accuracy: 0.7611 - val_loss: 0.5333
Epoch 7/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 28s 19ms/step - accuracy: 0.8473 - loss: 0.3355 - val_accuracy: 0.7593 - val_loss: 0.5770
Epoch 8/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 28s 18ms/step - accuracy: 0.8566 -

## Transformer

In [17]:
! pip install transformers
! pip install tf-keras



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 5.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.1/236.1 MB 9.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 7.5 MB/s eta 0:00:00a 0:00:01m
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 24.3.7
    Uninstalling flatbuffers-24.3.7:
      Successfully uninstalled flatbuffers-24.3.7
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.16.2
    Uninstalling tensorboard-2.16.2:
      Successfully uninstalled tensorboard-2.16.2
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.16.1
    Uninstalling tensorflow-2.16.1:
      Successfully uninstalled tensorflow-2.16.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. 

In [26]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Lambda
from transformers import TFBertModel, BertConfig

# 设置序列长度
sequence_length = 50

# 定义模型配置
config = BertConfig(
    vocab_size=30522,         # 词汇表大小
    hidden_size=768,          # 隐藏层维度
    num_hidden_layers=6,      # Transformer 编码层的数量
    num_attention_heads=12,   # 自注意力头的数量
    intermediate_size=2048,   # 前馈网络的大小
    hidden_dropout_prob=0.1,  # Dropout 的概率
    max_position_embeddings=50  # 最大序列长度
)

# 使用配置创建一个 Bert 模型
transformer_model = TFBertModel(config)

# 定义输入层，数据类型为 int32
input_ids = Input(shape=(sequence_length,), dtype=tf.int32, name="input_ids")

# 使用 Lambda 层将 input_ids 从 KerasTensor 转换为标准 TensorFlow 张量，并指定 output_shape
input_ids_tf = Lambda(lambda x: tf.convert_to_tensor(x, dtype=tf.int32), output_shape=(sequence_length,))(input_ids)

# 将输入传递给 Transformer 模型
bert_output = transformer_model(input_ids_tf)[0]

# 只取序列的第一个位置的输出进行分类（通常是 [CLS] token）
cls_token_output = bert_output[:, 0, :]

# 添加一个全连接层用于分类
output = Dense(1, activation="sigmoid")(cls_token_output)

# 定义完整的模型
model = Model(inputs=input_ids, outputs=output)

# 编译模型
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
              loss='binary_crossentropy', metrics=['accuracy'])

# 打印模型结构
model.summary()

# 训练模型的示例（假设你已经有X和y）
# history = model.fit(X, y, validation_split=0.2, epochs=3, batch_size=32)

# 评估模型（假设你有X_test和y_test）
# loss, accuracy = model.evaluate(X_test, y_test)
# print(f"Test Accuracy: {accuracy}")


ValueError: Exception encountered when calling layer 'tf_bert_model_5' (type TFBertModel).

Data of type <class 'keras.src.backend.common.keras_tensor.KerasTensor'> is not allowed only (<class 'tensorflow.python.framework.tensor.Tensor'>, <class 'bool'>, <class 'int'>, <class 'transformers.utils.generic.ModelOutput'>, <class 'tuple'>, <class 'list'>, <class 'dict'>, <class 'numpy.ndarray'>) is accepted for input_ids.

Call arguments received by layer 'tf_bert_model_5' (type TFBertModel):
  • input_ids=<KerasTensor shape=(None, 50), dtype=float32, sparse=False, name=keras_tensor_156>
  • attention_mask=None
  • token_type_ids=None
  • position_ids=None
  • head_mask=None
  • inputs_embeds=None
  • encoder_hidden_states=None
  • encoder_attention_mask=None
  • past_key_values=None
  • use_cache=None
  • output_attentions=None
  • output_hidden_states=None
  • return_dict=None
  • training=False